# Experiment 052: Proper GNN Implementation

**Hypothesis**: The GNN benchmark achieved MSE 0.0039 on this exact dataset. A proper GNN with Graph Attention Networks can achieve much better generalization than MLP/LGBM/GP ensembles.

**Key differences from exp_040 (failed GNN attempt):**
1. Full CV evaluation (all 24 folds for single, 13 for mixtures)
2. More training epochs (200 instead of 50)
3. Proper molecular graph construction from SMILES
4. Learned solvent embeddings combined with graph features
5. Arrhenius kinetics features

**Architecture:**
- Convert SMILES to molecular graphs using RDKit
- Use GAT layers to learn solvent representations
- Combine with Arrhenius features
- MLP head for prediction

In [1]:
import sys
sys.path.insert(0, '/home/code/experiments/049_manual_ood_handling')

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

# PyTorch Geometric
import torch_geometric
from torch_geometric.data import Data, Batch
from torch_geometric.nn import GATConv, global_mean_pool

# RDKit for molecular graphs
from rdkit import Chem
from rdkit.Chem import AllChem

print(f"PyTorch Geometric version: {torch_geometric.__version__}")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

PyTorch Geometric version: 2.7.0
PyTorch version: 2.2.0+cu118
CUDA available: True
GPU: NVIDIA A100-SXM4-80GB


In [2]:
# Load data
from utils_local import load_data, load_features, generate_leave_one_out_splits, generate_leave_one_ramp_out_splits

print("Loading data...")
X_single_raw, Y_single = load_data("single_solvent")
X_full_raw, Y_full = load_data("full")

print(f"Single solvent: {X_single_raw.shape}, Mixtures: {X_full_raw.shape}")

# Load features
spange = load_features("spange_descriptors")
drfp = load_features("drfps_catechol")
smiles_df = pd.read_csv('/home/data/smiles_lookup.csv', index_col=0)

print(f"Spange: {spange.shape}, DRFP: {drfp.shape}")
print(f"SMILES available for {len(smiles_df)} solvents")

Loading data...
Single solvent: (656, 3), Mixtures: (1227, 5)
Spange: (26, 13), DRFP: (24, 2048)
SMILES available for 26 solvents


In [3]:
# Molecular graph construction from SMILES
def smiles_to_graph(smiles):
    """
    Convert SMILES to PyTorch Geometric Data object.
    Node features: atomic number, degree, formal charge, hybridization, aromaticity
    Edge features: bond type, is_conjugated, is_in_ring
    """
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None
    
    # Node features
    atom_features = []
    for atom in mol.GetAtoms():
        features = [
            atom.GetAtomicNum(),
            atom.GetDegree(),
            atom.GetFormalCharge(),
            int(atom.GetHybridization()),
            int(atom.GetIsAromatic()),
            atom.GetTotalNumHs(),
            int(atom.IsInRing()),
        ]
        atom_features.append(features)
    
    x = torch.tensor(atom_features, dtype=torch.float)
    
    # Edge index and features
    edge_index = []
    edge_attr = []
    for bond in mol.GetBonds():
        i = bond.GetBeginAtomIdx()
        j = bond.GetEndAtomIdx()
        
        # Add both directions
        edge_index.append([i, j])
        edge_index.append([j, i])
        
        # Bond features
        bond_features = [
            float(bond.GetBondTypeAsDouble()),
            int(bond.GetIsConjugated()),
            int(bond.IsInRing()),
        ]
        edge_attr.append(bond_features)
        edge_attr.append(bond_features)  # Same for both directions
    
    if len(edge_index) == 0:
        # Single atom molecule
        edge_index = torch.zeros((2, 0), dtype=torch.long)
        edge_attr = torch.zeros((0, 3), dtype=torch.float)
    else:
        edge_index = torch.tensor(edge_index, dtype=torch.long).t().contiguous()
        edge_attr = torch.tensor(edge_attr, dtype=torch.float)
    
    return Data(x=x, edge_index=edge_index, edge_attr=edge_attr)

# Build graph cache for all solvents
print("Building molecular graphs for all solvents...")
solvent_graphs = {}
for solvent_name, row in smiles_df.iterrows():
    smiles = row['solvent smiles']
    graph = smiles_to_graph(smiles)
    if graph is not None:
        solvent_graphs[solvent_name] = graph
        print(f"  {solvent_name}: {graph.x.shape[0]} atoms, {graph.edge_index.shape[1]//2} bonds")
    else:
        print(f"  {solvent_name}: FAILED to parse SMILES '{smiles}'")

print(f"\nSuccessfully built graphs for {len(solvent_graphs)} solvents")

Building molecular graphs for all solvents...
  Cyclohexane: 6 atoms, 6 bonds
  Ethyl Acetate: 6 atoms, 5 bonds
  Acetic Acid: 4 atoms, 3 bonds
  2-Methyltetrahydrofuran [2-MeTHF]: 6 atoms, 6 bonds
  1,1,1,3,3,3-Hexafluoropropan-2-ol: 10 atoms, 9 bonds
  IPA [Propan-2-ol]: 4 atoms, 3 bonds
  Ethanol: 3 atoms, 2 bonds
  Methanol: 2 atoms, 1 bonds
  Ethylene Glycol [1,2-Ethanediol]: 4 atoms, 3 bonds
  Acetonitrile: 3 atoms, 2 bonds
  Water: 1 atoms, 0 bonds
  Diethyl Ether [Ether]: 5 atoms, 4 bonds
  MTBE [tert-Butylmethylether]: 6 atoms, 5 bonds
  Dimethyl Carbonate: 6 atoms, 5 bonds
  tert-Butanol [2-Methylpropan-2-ol]: 5 atoms, 4 bonds
  DMA [N,N-Dimethylacetamide]: 6 atoms, 5 bonds
  2,2,2-Trifluoroethanol: 6 atoms, 5 bonds
  Dihydrolevoglucosenone (Cyrene): 9 atoms, 10 bonds
  Decanol: 11 atoms, 10 bonds
  Butanone [MEK]: 5 atoms, 4 bonds
  Ethyl Lactate: 8 atoms, 7 bonds
  Methyl Propionate: 6 atoms, 5 bonds
  THF [Tetrahydrofuran]: 5 atoms, 5 bonds
  Water.Acetonitrile: 4 atoms, 2

In [4]:
# Prepare datasets with features
def prepare_single_solvent_dataset(X_raw, spange, drfp):
    """Prepare single solvent dataset with all features"""
    solvent_name = X_raw['SOLVENT NAME'].values
    spange_features = spange.loc[solvent_name].values
    drfp_features = drfp.loc[solvent_name].values
    time = X_raw['Residence Time'].values
    temp = X_raw['Temperature'].values
    
    spange_cols = spange.columns.tolist()
    drfp_cols = [f'DRFP_{i}' for i in range(drfp.shape[1])]
    
    df = pd.DataFrame(spange_features, columns=spange_cols)
    df_drfp = pd.DataFrame(drfp_features, columns=drfp_cols)
    df = pd.concat([df, df_drfp], axis=1)
    df['TEMPERATURE'] = temp
    df['TIME'] = time
    df['SOLVENT NAME'] = solvent_name
    
    return df

def prepare_mixture_dataset(X_raw, spange, drfp):
    """Prepare mixture dataset with all features"""
    solvent_a = X_raw['SOLVENT A NAME'].values
    solvent_b = X_raw['SOLVENT B NAME'].values
    solvent_b_pct = X_raw['SolventB%'].values / 100.0
    
    spange_a = spange.loc[solvent_a].values
    spange_b = spange.loc[solvent_b].values
    spange_mix = (1 - solvent_b_pct[:, None]) * spange_a + solvent_b_pct[:, None] * spange_b
    
    drfp_a = drfp.loc[solvent_a].values
    drfp_b = drfp.loc[solvent_b].values
    drfp_mix = (1 - solvent_b_pct[:, None]) * drfp_a + solvent_b_pct[:, None] * drfp_b
    
    solvent_name = [f"{a}.{b}" for a, b in zip(solvent_a, solvent_b)]
    time = X_raw['Residence Time'].values
    temp = X_raw['Temperature'].values
    
    spange_cols = spange.columns.tolist()
    drfp_cols = [f'DRFP_{i}' for i in range(drfp.shape[1])]
    
    df = pd.DataFrame(spange_mix, columns=spange_cols)
    df_drfp = pd.DataFrame(drfp_mix, columns=drfp_cols)
    df = pd.concat([df, df_drfp], axis=1)
    df['TEMPERATURE'] = temp
    df['TIME'] = time
    df['SOLVENT NAME'] = solvent_name
    df['SOLVENT A NAME'] = solvent_a
    df['SOLVENT B NAME'] = solvent_b
    df['SolventB%'] = X_raw['SolventB%'].values
    
    return df

X_single = prepare_single_solvent_dataset(X_single_raw, spange, drfp)
X_mix = prepare_mixture_dataset(X_full_raw, spange, drfp)

print(f"Single solvent dataset: {X_single.shape}")
print(f"Mixture dataset: {X_mix.shape}")

Single solvent dataset: (656, 2064)
Mixture dataset: (1227, 2067)


In [5]:
# GNN Model with GAT layers
class SolventGNN(nn.Module):
    """
    Graph Attention Network for solvent representation learning.
    """
    def __init__(self, node_features=7, hidden_dim=64, output_dim=32, heads=4, dropout=0.2):
        super().__init__()
        
        # GAT layers
        self.gat1 = GATConv(node_features, hidden_dim, heads=heads, dropout=dropout)
        self.gat2 = GATConv(hidden_dim * heads, hidden_dim, heads=heads, dropout=dropout)
        self.gat3 = GATConv(hidden_dim * heads, output_dim, heads=1, concat=False, dropout=dropout)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        
        # GAT layers with residual-like connections
        x = F.elu(self.gat1(x, edge_index))
        x = self.dropout(x)
        x = F.elu(self.gat2(x, edge_index))
        x = self.dropout(x)
        x = self.gat3(x, edge_index)
        
        # Global mean pooling
        x = global_mean_pool(x, batch)
        
        return x

class GNNYieldPredictor(nn.Module):
    """
    Full model: GNN for solvent + MLP for prediction.
    """
    def __init__(self, gnn_output_dim=32, arrhenius_dim=5, hidden_dim=64, dropout=0.2):
        super().__init__()
        
        self.gnn = SolventGNN(output_dim=gnn_output_dim, dropout=dropout)
        
        # MLP head
        combined_dim = gnn_output_dim + arrhenius_dim
        self.mlp = nn.Sequential(
            nn.Linear(combined_dim, hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.BatchNorm1d(hidden_dim // 2),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim // 2, 3),
            nn.Sigmoid()
        )
        
    def forward(self, graph_batch, arrhenius_features):
        # Get GNN embeddings
        gnn_emb = self.gnn(graph_batch)
        
        # Combine with Arrhenius features
        combined = torch.cat([gnn_emb, arrhenius_features], dim=1)
        
        # Predict
        return self.mlp(combined)

print("GNN models defined")

GNN models defined


In [6]:
# GNN Model wrapper for CV
class GNNModel:
    """
    GNN model wrapper that handles training and prediction.
    """
    def __init__(self, epochs=200, lr=1e-3, weight_decay=1e-4, dropout=0.2):
        self.epochs = epochs
        self.lr = lr
        self.weight_decay = weight_decay
        self.dropout = dropout
        
        self.model = None
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.arrhenius_scaler = StandardScaler()
    
    def _get_arrhenius_features(self, X_data):
        """Extract Arrhenius kinetics features"""
        T = X_data['TEMPERATURE'].values
        t = X_data['TIME'].values
        T_kelvin = T + 273.15
        inv_T = 1000.0 / T_kelvin
        ln_t = np.log(t + 1e-6)
        interaction = inv_T * ln_t
        return np.column_stack([T, t, inv_T, ln_t, interaction])
    
    def _get_solvent_graphs(self, X_data):
        """Get molecular graphs for solvents"""
        graphs = []
        solvent_names = X_data['SOLVENT NAME'].values
        
        for solvent in solvent_names:
            # Handle mixture solvents
            if '.' in solvent:
                # For mixtures, use the first solvent's graph (simplified)
                parts = solvent.split('.')
                base_solvent = parts[0]
            else:
                base_solvent = solvent
            
            if base_solvent in solvent_graphs:
                graphs.append(solvent_graphs[base_solvent])
            else:
                # Fallback: create a dummy graph
                print(f"Warning: No graph for {base_solvent}, using dummy")
                dummy = Data(x=torch.zeros((1, 7)), edge_index=torch.zeros((2, 0), dtype=torch.long))
                graphs.append(dummy)
        
        return graphs
    
    def fit(self, X_train, Y_train):
        """Train the GNN model"""
        # Get features
        arrhenius = self._get_arrhenius_features(X_train)
        arrhenius_scaled = self.arrhenius_scaler.fit_transform(arrhenius)
        graphs = self._get_solvent_graphs(X_train)
        
        Y_values = Y_train.values
        
        # Create model
        self.model = GNNYieldPredictor(dropout=self.dropout).to(self.device)
        optimizer = torch.optim.Adam(self.model.parameters(), lr=self.lr, weight_decay=self.weight_decay)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.5, patience=20)
        criterion = nn.HuberLoss()
        
        # Convert to tensors
        arrhenius_tensor = torch.FloatTensor(arrhenius_scaled).to(self.device)
        Y_tensor = torch.FloatTensor(Y_values).to(self.device)
        
        # Training loop
        self.model.train()
        for epoch in range(self.epochs):
            optimizer.zero_grad()
            
            # Batch graphs
            graph_batch = Batch.from_data_list(graphs).to(self.device)
            
            # Forward pass
            pred = self.model(graph_batch, arrhenius_tensor)
            loss = criterion(pred, Y_tensor)
            
            # Backward pass
            loss.backward()
            optimizer.step()
            scheduler.step(loss)
        
        return self
    
    def predict(self, X_test):
        """Predict using the trained model"""
        # Get features
        arrhenius = self._get_arrhenius_features(X_test)
        arrhenius_scaled = self.arrhenius_scaler.transform(arrhenius)
        graphs = self._get_solvent_graphs(X_test)
        
        # Convert to tensors
        arrhenius_tensor = torch.FloatTensor(arrhenius_scaled).to(self.device)
        
        # Predict
        self.model.eval()
        with torch.no_grad():
            graph_batch = Batch.from_data_list(graphs).to(self.device)
            pred = self.model(graph_batch, arrhenius_tensor).cpu().numpy()
        
        return np.clip(pred, 0, 1)

print("GNNModel wrapper defined")

GNNModel wrapper defined


In [ ]:
# Run CV for single solvents with GNN
print("Running Single Solvent CV with GNN Model...")
print("="*60)

splits = list(generate_leave_one_out_splits(X_single, Y_single))
print(f"Number of folds: {len(splits)}")

solvent_errors_gnn = {}
all_preds_gnn = []
all_true_gnn = []

for fold_idx, (train_idx, test_idx) in enumerate(splits):
    X_train = X_single.iloc[train_idx]
    Y_train = Y_single.iloc[train_idx]
    X_test = X_single.iloc[test_idx]
    Y_test = Y_single.iloc[test_idx]
    
    test_solvent = X_test['SOLVENT NAME'].iloc[0]
    
    # Train model
    model = GNNModel(epochs=200, lr=1e-3, dropout=0.2)
    model.fit(X_train, Y_train)
    
    # Predict
    preds = model.predict(X_test)
    
    # Calculate MSE
    mse = np.mean((preds - Y_test.values) ** 2)
    solvent_errors_gnn[test_solvent] = mse
    
    all_preds_gnn.append(preds)
    all_true_gnn.append(Y_test.values)
    
    print(f"Fold {fold_idx+1:2d}: {test_solvent:45s} MSE = {mse:.6f}")

all_preds_gnn = np.vstack(all_preds_gnn)
all_true_gnn = np.vstack(all_true_gnn)
single_mse_gnn = np.mean((all_preds_gnn - all_true_gnn) ** 2)
single_std_gnn = np.std([solvent_errors_gnn[s] for s in solvent_errors_gnn])

print(f"\nGNN Single Solvent CV MSE: {single_mse_gnn:.6f} +/- {single_std_gnn:.6f}")

In [ ]:
# Run CV for mixtures with GNN
print("\n" + "="*60)
print("Running Mixture CV with GNN Model...")
print("="*60)

mix_splits = list(generate_leave_one_ramp_out_splits(X_mix, Y_full))
print(f"Number of folds: {len(mix_splits)}")

mix_errors_gnn = {}
mix_preds_gnn = []
mix_true_gnn = []

for fold_idx, (train_idx, test_idx) in enumerate(mix_splits):
    X_train = X_mix.iloc[train_idx]
    Y_train = Y_full.iloc[train_idx]
    X_test = X_mix.iloc[test_idx]
    Y_test = Y_full.iloc[test_idx]
    
    test_mixture = X_test['SOLVENT NAME'].iloc[0]
    
    # Train model
    model = GNNModel(epochs=200, lr=1e-3, dropout=0.2)
    model.fit(X_train, Y_train)
    
    # Predict
    preds = model.predict(X_test)
    
    # Calculate MSE
    mse = np.mean((preds - Y_test.values) ** 2)
    mix_errors_gnn[test_mixture] = mse
    
    mix_preds_gnn.append(preds)
    mix_true_gnn.append(Y_test.values)
    
    print(f"Fold {fold_idx+1:2d}: {test_mixture:55s} MSE = {mse:.6f}")

mix_preds_gnn = np.vstack(mix_preds_gnn)
mix_true_gnn = np.vstack(mix_true_gnn)
mix_mse_gnn = np.mean((mix_preds_gnn - mix_true_gnn) ** 2)
mix_std_gnn = np.std([mix_errors_gnn[s] for s in mix_errors_gnn])

print(f"\nGNN Mixture CV MSE: {mix_mse_gnn:.6f} +/- {mix_std_gnn:.6f}")

In [ ]:
# Calculate overall CV score
print("\n" + "="*60)
print("GNN Model Overall Results")
print("="*60)

n_single = len(all_true_gnn)
n_mix = len(mix_true_gnn)
n_total = n_single + n_mix

overall_mse_gnn = (n_single * single_mse_gnn + n_mix * mix_mse_gnn) / n_total

print(f"\nSingle Solvent CV MSE: {single_mse_gnn:.6f} +/- {single_std_gnn:.6f} (n={n_single})")
print(f"Mixture CV MSE: {mix_mse_gnn:.6f} +/- {mix_std_gnn:.6f} (n={n_mix})")
print(f"Overall CV MSE: {overall_mse_gnn:.6f}")

print(f"\nBaseline (exp_030): CV = 0.008298")
print(f"GNN Benchmark: MSE = 0.0039")
print(f"Improvement vs baseline: {(0.008298 - overall_mse_gnn) / 0.008298 * 100:.1f}%")

if overall_mse_gnn < 0.008298:
    print("\n✓ BETTER than baseline!")
else:
    print("\n✗ WORSE than baseline.")

In [ ]:
# Final Summary
print("\n" + "="*60)
print("EXPERIMENT 052 SUMMARY")
print("="*60)

print(f"\nGNN Model:")
print(f"  Single Solvent CV: {single_mse_gnn:.6f}")
print(f"  Mixture CV: {mix_mse_gnn:.6f}")
print(f"  Overall CV: {overall_mse_gnn:.6f}")
print(f"  vs Baseline (exp_030): {(overall_mse_gnn - 0.008298) / 0.008298 * 100:+.1f}%")

print("\nKey insights:")
print("1. GNN uses molecular graph structure from SMILES")
print("2. GAT layers learn attention-weighted node aggregation")
print("3. Combined with Arrhenius kinetics features")

if overall_mse_gnn < 0.008298:
    print("\nCONCLUSION: GNN IMPROVES overall CV!")
    print("This is a fundamentally different approach that may change the CV-LB relationship.")
    print("Consider submitting to test the new relationship.")
else:
    print("\nCONCLUSION: GNN does NOT improve overall CV.")
    print("The GNN benchmark's success may be due to different implementation details.")

print(f"\nRemaining submissions: 5")
print(f"Best model: exp_030 (GP 0.15 + MLP 0.55 + LGBM 0.3) with CV 0.008298, LB 0.0877")